1) Importamos las librerias

In [74]:
import pandas as pd
import numpy as np
import json
import ast
import re

1) Importamos los json

In [32]:
df_games = pd.read_json(r'Datasets\output_steam_games.json' ,lines=True)

In [49]:
data_list = []        # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_user_reviews.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_user_reviews = pd.DataFrame(data_list)

In [34]:
data_list = []   # Lista para almacenar los diccionarios JSON de cada línea

file_path = r"Datasets\australian_users_items.json"

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_items = pd.DataFrame(data_list)

#revisar que hace está funcion xd

Desanidamos los datasets anidados

In [ ]:
df_user_reviews_desc = df_user_reviews.explode(['reviews']) #descomponemos la columna reviews
df_reviews_series = df_user_reviews_desc['reviews'].apply(pd.Series) #creamos una serie a partir de ella
df_user_reviews = pd.concat([df_user_reviews_desc, df_reviews_series], axis=1) #creamos el dataframe definitivo 
                                                                            #y despues borramos la columna que quedó de más
df_user_reviews.drop('reviews', axis=1, inplace = True)     #al estar agregada al dataframe,ya la podemos borrar

In [129]:
df_user_items_desc = df_user_items.explode(["items"])
df_user_items_desc2 = pd.json_normalize(df_user_items_desc['items']).set_index(df_user_items_desc['items'].index)
df_user_items= pd.concat([df_user_items_desc2, df_user_items_desc], axis=1)
df_user_items.drop('items', axis=1, inplace = True)

In [120]:
df_games = df_games.explode(["genres"])
df_games = df_games.explode(["tags"])
df_games = df_games.explode(["specs"])

In [141]:
df_user_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Borramos todas las filas que esten totalmente vacias y duplicados

In [134]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 2088826
la cantidad de filas de items son 20629642
la cantidad de filas de reviews son 59333


In [136]:
df_games = df_games.dropna(how ="all")
df_user_reviews = df_user_reviews.dropna(how ="all")
df_user_items = df_user_items.dropna(how ="all")
df_user_reviews.drop(0, axis=1, inplace = True)             #borramos la columna vacia

In [138]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 2088826
la cantidad de filas de items son 20629642
la cantidad de filas de reviews son 59333


In [139]:
df_games = df_games.drop_duplicates()
df_user_reviews = df_user_reviews.drop_duplicates()
df_user_items = df_user_items.drop_duplicates()

In [140]:
print("la cantidad de filas de games son", df_games.shape[0])
print("la cantidad de filas de items son", df_user_items.shape[0])
print("la cantidad de filas de reviews son", df_user_reviews.shape[0])

la cantidad de filas de games son 2088826
la cantidad de filas de items son 5110819
la cantidad de filas de reviews son 58459
